In [3]:
import torch 
import torchvision
import numpy as np
import os 
import pandas as pd 

from PIL import Image

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [4]:
path="E:\\datasets\\yoga pose estimation\\"

In [5]:
test_df=pd.read_pickle(path+"datasettest_df_yoga.pkl")

In [6]:
train_df=pd.read_pickle(path+"datasettrain_yoga.pkl")

In [7]:
test_df

,Label,Image
0,eka pada rajakapotasana,<PIL.PngImagePlugin.PngImageFile image mode=RG...
1,savasana,<PIL.PngImagePlugin.PngImageFile image mode=RG...
2,ardha pincha mayurasana,<PIL.PngImagePlugin.PngImageFile image mode=RG...
3,adho mukha vriksasana,<PIL.PngImagePlugin.PngImageFile image mode=RG...
4,bhairavasana,<PIL.PngImagePlugin.PngImageFile image mode=RG...
...,...,...
1121,garudasana,<PIL.PngImagePlugin.PngImageFile image mode=RG...
1122,viparita karani,<PIL.PngImagePlugin.PngImageFile image mode=RG...
1123,urdhva hastasana,<PIL.PngImagePlugin.PngImageFile image mode=RG...
1124,utthita ashwa sanchalanasana,<PIL.PngImagePlugin.PngImageFile image mode=RG...


In [8]:
from torch.utils.data import Dataset ,DataLoader
from torchvision import transforms
from sklearn.preprocessing import LabelEncoder

In [9]:
train_df

,Label,Image
0,eka pada rajakapotasana,<PIL.PngImagePlugin.PngImageFile image mode=RG...
1,bharadvajasana i,<PIL.PngImagePlugin.PngImageFile image mode=RG...
2,bhujapidasana,<PIL.PngImagePlugin.PngImageFile image mode=RG...
3,urdhva mukha svanasana,<PIL.PngImagePlugin.PngImageFile image mode=RG...
4,anantasana,<PIL.PngImagePlugin.PngImageFile image mode=RG...
...,...,...
4496,setu bandha sarvangasana,<PIL.PngImagePlugin.PngImageFile image mode=RG...
4497,viparita karani,<PIL.PngImagePlugin.PngImageFile image mode=RG...
4498,paripurna navasana,<PIL.PngImagePlugin.PngImageFile image mode=RG...
4499,setu bandha sarvangasana,<PIL.PngImagePlugin.PngImageFile image mode=RG...


In [10]:
combined_df = pd.concat([train_df, test_df], ignore_index=True)

print(combined_df)

                             Label  \
0          eka pada rajakapotasana   
1                 bharadvajasana i   
2                    bhujapidasana   
3           urdhva mukha svanasana   
4                       anantasana   
...                            ...   
5622                    garudasana   
5623               viparita karani   
5624              urdhva hastasana   
5625  utthita ashwa sanchalanasana   
5626                      malasana   

                                                  Image  
0     <PIL.PngImagePlugin.PngImageFile image mode=RG...  
1     <PIL.PngImagePlugin.PngImageFile image mode=RG...  
2     <PIL.PngImagePlugin.PngImageFile image mode=RG...  
3     <PIL.PngImagePlugin.PngImageFile image mode=RG...  
4     <PIL.PngImagePlugin.PngImageFile image mode=RG...  
...                                                 ...  
5622  <PIL.PngImagePlugin.PngImageFile image mode=RG...  
5623  <PIL.PngImagePlugin.PngImageFile image mode=RG...  
5624  <PIL.PngImagePl

In [11]:
label_encoder=LabelEncoder()

In [12]:
combined_df["Label"]=label_encoder.fit_transform(combined_df["Label"])

In [13]:
num_classes=combined_df['Label'].nunique()

In [14]:
num_classes

107

In [15]:
dataframe=combined_df

In [16]:
import torch 
from torch.utils.data import Dataset
from PIL import Image

In [17]:
class CustomDataset(Dataset):
    def __init__ (self,dataframe,transform=None):
        self.data=dataframe
        self.transform=transform
    def __len__(self):

        return len(self.data)

        
    def __getitem__(self,index):

        label=self.data.iloc[index]["Label"]

        image=self.data.iloc[index]["Image"]

        if self.transform:
            image=self.transform(image)


        return image, label






In [18]:
from torchvision import transforms

In [23]:
transform = transforms.Compose([
    transforms.Resize((299, 299)),  #  INCEPTION Network
    transforms.ToTensor(),  # Görüntüyü tensöre çeviriyoruz
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  
                         std=[0.229, 0.224, 0.225])
])

In [24]:
dataset=CustomDataset(dataframe=dataframe,transform=transform)

In [25]:
image,label=dataset[0]

In [26]:
image.shape

torch.Size([3, 299, 299])

In [41]:
labels = dataset.data['Label'].values[:len(dataset)]  

indices = list(range(len(dataset)))
train_indices, test_indices = train_test_split(indices, test_size=0.2, stratify=labels)

train_dataset = torch.utils.data.Subset(dataset, train_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [27]:
import torchvision.models as models

model= models.inception_v3(pretrained=True)

c:\Users\ninja\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ninja\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to C:\Users\ninja/.cache\torch\hub\checkpoints\inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:09<00:00, 11.8MB/s] 


In [28]:
from torchsummary import summary 

In [30]:
model

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [32]:
import torch.nn as nn

In [33]:
model.fc =nn.Linear(model.fc.in_features,num_classes)

In [36]:
if torch.cuda.is_available():
    device=torch.device("cuda")

In [38]:
model.to(device)

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [39]:
summary(model,input_size=(3,299,299))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 149, 149]             864
       BatchNorm2d-2         [-1, 32, 149, 149]              64
       BasicConv2d-3         [-1, 32, 149, 149]               0
            Conv2d-4         [-1, 32, 147, 147]           9,216
       BatchNorm2d-5         [-1, 32, 147, 147]              64
       BasicConv2d-6         [-1, 32, 147, 147]               0
            Conv2d-7         [-1, 64, 147, 147]          18,432
       BatchNorm2d-8         [-1, 64, 147, 147]             128
       BasicConv2d-9         [-1, 64, 147, 147]               0
        MaxPool2d-10           [-1, 64, 73, 73]               0
           Conv2d-11           [-1, 80, 73, 73]           5,120
      BatchNorm2d-12           [-1, 80, 73, 73]             160
      BasicConv2d-13           [-1, 80, 73, 73]               0
           Conv2d-14          [-1, 192,

In [40]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [44]:
num_epochs = 1

for epoch in range(num_epochs):
    model.train()  
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        # Forward pass
        outputs = model(images)
        labels=labels.long()
        #print(outputs.dtype)
        #print(labels.dtype)
        # 2 loss in inception
        loss1 = criterion(outputs.logits, labels)  # main loss
        loss2 = criterion(outputs.aux_logits, labels)  # helpful loss
        loss = loss1 + 0.4 * loss2  #total loos

        #loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


Epoch [1/1], Loss: 2.9318


In [45]:
model.eval()   
correct = 0
total = 0

with torch.no_grad():   
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')

Accuracy: 46.27%
